# Deploy Image to AWS Lambda

## Dockerfile

## predict.py

## Build Image (local)

In [9]:
#!docker build -t hairstyle-predictor-lambda .

## Run Container (Run Locally - AWS Lambda Runtime)

In [11]:
#!docker run -p 9000:8080 hairstyle-predictor-lambda

## Invoke Lambda Function (Testing with JSON)

In [ ]:
!curl -X POST "http://localhost:9000/2015-03-31/functions/function/invocations" -d '{"image_url": "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"}' | jq .

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   139  100    51  100    88     27     47  0:00:01  0:00:01 --:--:--    74
{
  "statusCode": 200,
  "body": {
    "prediction": 0.9131
  }
}


In [4]:
# Authenticate Docker to AWS ECR, use the AWS CLI to authenticate Docker client

!aws ecr get-login-password --region ap-southeast-3 | docker login --username AWS --password-stdin 734800375959.dkr.ecr.ap-southeast-3.amazonaws.com 


WARNING! Your password will be stored unencrypted in /home/hduser/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [5]:
# Create repository
!aws ecr create-repository --repository-name hairstyle-predictor-lambda


{
    "repository": {
        "repositoryArn": "arn:aws:ecr:ap-southeast-3:734800375959:repository/hairstyle-predictor-lambda",
        "registryId": "734800375959",
        "repositoryName": "hairstyle-predictor-lambda",
        "repositoryUri": "734800375959.dkr.ecr.ap-southeast-3.amazonaws.com/hairstyle-predictor-lambda",
        "createdAt": 1733902457.806,
        "imageTagMutability": "MUTABLE",
        "imageScanningConfiguration": {
            "scanOnPush": false
        },
        "encryptionConfiguration": {
            "encryptionType": "AES256"
        }
    }
}


In [7]:
# Tag image to AWS
!docker tag hairstyle-predictor-lambda 734800375959.dkr.ecr.ap-southeast-3.amazonaws.com/hairstyle-predictor-lambda:latest


In [8]:
# Push image
!docker push 734800375959.dkr.ecr.ap-southeast-3.amazonaws.com/hairstyle-predictor-lambda:latest

The push refers to repository [734800375959.dkr.ecr.ap-southeast-3.amazonaws.com/hairstyle-predictor-lambda]

5e61e1b4: Preparing 
92ab2484: Preparing 
393f615f: Preparing 
bf18a086: Preparing 
87eabbbc: Preparing 
096288d2: Preparing 
b0f618e4: Preparing 
3c040d49: Preparing 
a108b9e3: Preparing 
e61e1b4: Pushed   2.228GB/2.211GB[10APushing  2.169GB/2.211GBlatest: digest: sha256:65e7116b30e2a3f2590e914224700f0016673696826ca4215ff25dab52b50f20 size: 2420


In [19]:
# Create AWS Lambda Function:
!aws lambda create-function \
  --function-name hairstyle-predictor \
  --package-type Image \
  --code ImageUri=734800375959.dkr.ecr.ap-southeast-3.amazonaws.com/hairstyle-predictor-lambda:latest \
  --role arn:aws:iam::734800375959:role/hairstyle-predictor-role


{
    "FunctionName": "hairstyle-predictor",
    "FunctionArn": "arn:aws:lambda:ap-southeast-3:734800375959:function:hairstyle-predictor",
    "Role": "arn:aws:iam::734800375959:role/hairstyle-predictor-role",
    "CodeSize": 0,
    "Description": "",
    "Timeout": 3,
    "MemorySize": 128,
    "LastModified": "2024-12-11T08:17:04.823+0000",
    "CodeSha256": "65e7116b30e2a3f2590e914224700f0016673696826ca4215ff25dab52b50f20",
    "Version": "$LATEST",
    "TracingConfig": {
        "Mode": "PassThrough"
    },
    "RevisionId": "0a6727fb-8a0e-4c5b-9812-db1b97d82f3a",
    "State": "Pending",
    "StateReason": "The function is being created.",
    "StateReasonCode": "Creating",
    "PackageType": "Image",
    "Architectures": [
        "x86_64"
    ]
}


## API Gateway Integration


## Test Using Curl - Local Lambda Runtime

Send a JSON request to the local Lambda runtime:

In [30]:
!curl -X POST "http://localhost:9000/2015-03-31/functions/function/invocations" \
-d '{"image_url": "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"}'


{"statusCode": 200, "body": {"prediction": 0.9131}}

## Test Using Curl - AWS Lambda - API Gateway 

In [31]:
!curl -X POST https://b63orciqnk.execute-api.ap-southeast-3.amazonaws.com/test/predict \
  -H "Content-Type: application/json" \
  -H "x-api-key: ~/.ssh/ml-zoomcamp-gar-key.pem" \
  -d '{"image_url": "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"}'

{"statusCode": 200, "body": {"prediction": 0.9131}}